## Baseline

Start with simple data pipeline and make a crappy classifier. This will just the simplest thing that works.

Eliminate all unlabeled profiles, using 80/20 train/test split without consideration for
singleton labels or good stratification of samples, using just composition words as features:

Naive Bayes (default config) 54%/45% train/test

Linear SVM (default config) 96%/51% 

In [1]:
import numpy as np
import profile_classification as pc
raw=pc.load_raw_profile_data()
labeled=pc.get_labeled_profiles(raw)
data=pc.make_train_test_data(labeled)
#print [(k,v[:10]) for k,v in data.items()]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


X_train,y_train=data['X_train'],data['y_train']
X_test,y_test=data['X_test'],data['y_test']

vect=CountVectorizer(min_df=1,ngram_range=(1,1))
svd=TruncatedSVD()
X_v_train=vect.fit_transform([str(d) for d in X_train])
X_v_test=vect.transform([str(d) for d in X_test])

mNB=MultinomialNB()
svc=LinearSVC()
dt=DecisionTreeClassifier(min_samples_leaf=1)

for clf in (mNB,svc,dt,): 
    clf.fit(X_v_train,y_train)
    y_pred=clf.predict(X_v_test)
    print np.sum(y_pred==y_test)/float(len(y_test))
    #print classification_report(y_test,y_pred)
    

0.449204406365
0.514075887393
0.481028151775


In [100]:
r_w=raw.dropna(subset=['Composition'])
r_w=r_w['Composition']
vect.fit(r_w)


CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [101]:
print len(vect.get_feature_names())

9258
